# E-Commerce Data | Actual transactions from UK retailer

Refrence: 

Data right deserved for The UCI Machine Learning Repository has made this dataset containing actual transactions from 2010 and 2011. The dataset is maintained on their site, where it can be found by the title "Online Retail".

[Link1](https://archive.ics.uci.edu/dataset/352/online+retail)             [Link2](https://archive.ics.uci.edu/dataset/502/online+retail+ii)

"This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers."

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.float_format = "{:.3f}".format

## Importing Data

In [2]:
d1 = pd.read_excel('./Data/Online Retail.xlsx')
d2 = pd.read_excel('./Data/online_retail_II.xlsx')

d2.columns = d1.columns

In [3]:
data = pd.concat([d1, d2], ignore_index=True)
df = data.copy()

print(df.shape)
df.head()

(1067370, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom


## Deep Dive to Data
Doing an analysis on each column


| Variable Name | Role      | Type         | Description                                                                                      
|---------------|-----------|--------------|--------------------------------------------------------------------------------------------------
| InvoiceNo     | ID        | Categorical  | a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation | no        |
| StockCode     | ID        | Categorical  | a 5-digit integral number uniquely assigned to each distinct product                             
| Description   | Feature   | Categorical  | product name                                                                                    
| Quantity      | Feature   | Integer      | the quantities of each product (item) per transaction                                           
| InvoiceDate   | Feature   | Date         | the day and time when each transaction was generated                                            
| UnitPrice     | Feature   | Continuous   | product price per unit                                                                          
| CustomerID    | Feature   | Categorical  | a 5-digit integral number uniquely assigned to each customer                                     
| Country       | Feature   | Categorical  | the name of the country where each customer resides   



In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067370 entries, 0 to 1067369
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   InvoiceNo    1067370 non-null  object        
 1   StockCode    1067370 non-null  object        
 2   Description  1062988 non-null  object        
 3   Quantity     1067370 non-null  int64         
 4   InvoiceDate  1067370 non-null  datetime64[ns]
 5   UnitPrice    1067370 non-null  float64       
 6   CustomerID   824363 non-null   float64       
 7   Country      1067370 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 65.1+ MB


- we have null values in `Description` and `CustomerID` which is cool!

In [5]:
df.describe(include='all')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,1067370.000,1067370,1062988,1067370.000,1067370,1067370.000,824363.000,1067370
unique,53628.000,5305,5698,NaN,NaN,NaN,NaN,43
top,537434.000,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,NaN,NaN,NaN,United Kingdom
freq,1350.000,5829,5918,NaN,NaN,NaN,NaN,981330
mean,NaN,NaN,NaN,9.939,2011-01-02 21:13:27.819556096,4.649,15324.642,NaN
min,NaN,NaN,NaN,-80995.000,2009-12-01 07:45:00,-53594.360,12346.000,NaN
25%,NaN,NaN,NaN,1.000,2010-07-09 09:46:00,1.250,13975.000,NaN
50%,NaN,NaN,NaN,3.000,2010-12-07 15:28:00,2.100,15255.000,NaN
75%,NaN,NaN,NaN,10.000,2011-07-22 10:23:00,4.150,16797.000,NaN
max,NaN,NaN,NaN,80995.000,2011-12-09 12:50:00,38970.000,18287.000,NaN


In [6]:
(df.isnull().sum()/df.shape[0]*100)

InvoiceNo      0.000
StockCode      0.000
Description    0.411
Quantity       0.000
InvoiceDate    0.000
UnitPrice      0.000
CustomerID    22.767
Country        0.000
dtype: float64

In [7]:
print('duplicates %% : %.2f' % (df.duplicated().sum() / df.shape[0] * 100))
df.drop_duplicates(inplace=True)

duplicates % : 3.22


### Invoice No 

- First we split the 'C' represeting the canceled orders and Invoice No and make it clean. 
- Second, it seems there is a mistake in data for 6 rows which starts with 'A'. I replaced them with 'C' and I assumed it's an data entry error. 

In [8]:
IsCanceled = []
CleanInvoice = []

for index, row in df.iterrows(): 
   IsCanceled.append(str(row['InvoiceNo'])[0].isalpha())
   CleanInvoice.append(int(str(row['InvoiceNo']).replace('C', '').replace('A', '')))

df['IsCanceled'] = IsCanceled
df['InvoiceNo'] = CleanInvoice

In [9]:
#final check
for index, row in df.iterrows(): 
    value = row['InvoiceNo']

    if len(str(value)) != 6: 
        print('Mistake')
    if str(value).isnumeric: 
        pass
    else:
        print(index)

we have all of them in 6 digits! 

In [10]:
df[['InvoiceNo', 'IsCanceled']].describe(include='all')

,InvoiceNo,IsCanceled
count,1033035.000,1033035
unique,NaN,2
top,NaN,False
freq,NaN,1013925
mean,537574.872,NaN
std,26945.950,NaN
min,489434.000,NaN
25%,514558.000,NaN
50%,538053.000,NaN
75%,561473.000,NaN


In [11]:
DFCK1 = df.copy()
DFCK1.shape
#checkpoint 1

(1033035, 9)

### Price


In [12]:
#adding total price 
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,IsCanceled,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom,False,15.300
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,False,20.340
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom,False,22.000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,False,20.340
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,False,20.340


In [13]:
print('number of price = 0 is : ' , df[df['TotalPrice'] == 0 ].shape[0])
print(6010/df.shape[0]*100)

number of price = 0 is :  6014
0.5817808689928221


there are 6k row with 0 price, they are probably because of glitches in system which had lead to those adjustments and manuals (Just an hyphothesis)

less than 1 % of datas ==> I'll drop them.

In [14]:
df.drop(df[df['TotalPrice'] == 0 ].index, inplace=True)

In [15]:
df[df['UnitPrice'] <=  0].shape

(5, 10)

In [16]:
DFCK2 = df.copy()
DFCK2.shape
#checkpoint 1

(1027021, 10)

### Stock Code & Description


First let's see if the codes are legit to 5 digit number? 

In [17]:
temp = pd.DataFrame()
temp['code'] = df['StockCode']
temp['desc'] = df['Description']
temp['len'] = None

In [18]:
for index, row in temp.iterrows(): 
    temp['len'][index] = len(str(row['code']))
temp.groupby('len').agg(['count', 'nunique'])

code            desc        
      count nunique   count nunique
len                                
1      1665       5    1665       4
2       274       1     274       1
3      1418       1    1418       1
4      2112       3    2112       3
5    897765    3401  897765    3975
6    122113    1488  122113    1400
7      1336       8    1336       6
8        99      14      99      14
9        61       3      61       3
12      178       8     178       9

In [19]:
# checking each of the each category: 

Length = 4  #enter the number here 
temp[temp['len'] == Length ].iloc[:,0:2].drop_duplicates()['code'].values

array(['POST', 'PADS', 'CRUK'], dtype=object)

based on the length of each code, I did the analysis and arrived to this following strategy to clean the data. 

from 1 to 4 length and more than 7, the observation are summarized in: 

- `D` : Discount  --> negative price for Canceled order and positive for approved order
- `M` / `m` : Manual	--> seems to be some manual adjustments 
- `S`	: SAMPLES	--> Droping them! No clue what are they
- `B`	: Adjust bad debt	--> seems to be for accounting purposes, so we drop them 
- `C2` / `C3` : CARRIAGE   --> changed to Delivery cost, it's mostly 50 euros 
- `DOT` : 'DOTCOM POSTAGE' seems to indicate the amount spent by the customer on postage. --> changed to DeliveryCost
- `POST` : POSTAGE --> changed to DeliveryCost
- `GIFT` / gift_0001_20 : gifts, starting with gift_ ... 
- `CRUK` : CRUK Commission --> No info => drop it
- `ADJUST` / `ADJUST2` : The description is : "Adjustment by john on 26/01/2010 16" they are all happend on the same day, so there might be a problem on that day. ==> droping them
- `TEST002` / `TEST001`: starting with test, are tests --> drop them 
- `AMAZONFEE` : cost paied to amazoon apperantly  --> no idea about them so i'll drop them as well
- `BANK CHARGES` : appearantly they are income if order is not canceled. 


for the rest which are 5, 6, and 7, the strategy is as folllows: 

the first 5 charachter of 6 and 7 will be stored as codes, and the rest are `attribute`s which corresponds to color, type, etc. 
and finally I'll examine all 5 char codes to make sure they follow the rules. 



In [20]:
# Replace 'D' in the 'StockCode' column with 'Discount_____'
df.loc[df['StockCode'] == 'D', 'StockCode'] = 'Discount_____'

# Replace 'M' or 'm' in the 'StockCode' column with 'Manual_______'
df.loc[(df['StockCode'] == 'M') | (df['StockCode'] == 'm'), 'StockCode'] = 'Manual_______'

# Drop rows where 'StockCode' is 'S'
df.drop(df[df['StockCode'] == 'S'].index, inplace=True)

# Drop rows where 'StockCode' is 'B'
df.drop(df[df['StockCode'] == 'B'].index, inplace=True)

# Replace 'C2' in the 'StockCode' column with 'DeliveryCost_'
df.loc[df['StockCode'] == 'C2', 'StockCode'] = 'DeliveryCost_'

# Drop rows where 'StockCode' is 'C3'
df.drop(df[df['StockCode'] == 'C3'].index, inplace=True)

# Replace 'DOT' in the 'StockCode' column with 'DeliveryCost_'
df.loc[df['StockCode'] == 'DOT', 'StockCode'] = 'DeliveryCost_'

# Replace 'POST' in the 'StockCode' column with 'DeliveryCost_'
df.loc[df['StockCode'] == 'POST', 'StockCode'] = 'DeliveryCost_'

# Replace specific 'StockCode' values related to gifts or bank charges with 'Gift_________'
df.loc[df['StockCode'].isin(['GIFT', 'gift_0001_40', 'gift_0001_50', 'gift_0001_30',
                             'gift_0001_20', 'gift_0001_10', 'gift_0001_20', 'gift_0001_30', 
                             'gift_0001_10', 'BANK CHARGES', 'gift_0001_80', 'gift_0001_80',
                             'gift_0001_50', 'gift_0001_40', 'gift_0001_60', 'gift_0001_70',
                             'gift_0001_90', 'gift_0001_70', 'gift_0001_20']), 'StockCode'] = 'Gift_________'

# Drop rows where 'StockCode' is 'CRUK'
df.drop(df[df['StockCode'] == 'CRUK'].index, inplace=True)

# Drop rows where 'StockCode' is 'ADJUST' or 'ADJUST2'
df.drop(df[df['StockCode'].isin(['ADJUST', 'ADJUST2'])].index, inplace=True)

# Drop rows where 'StockCode' is 'TEST002' or 'TEST001'
df.drop(df[df['StockCode'].isin(['TEST002', 'TEST001'])].index, inplace=True)

# Drop rows where 'StockCode' is 'AMAZONFEE'
df.drop(df[df['StockCode'] == 'AMAZONFEE'].index, inplace=True)

In [21]:
# for 5, 6, and 7 char stockcodes
df['ProductAttribute'] = None

for i, row in df.iterrows(): 
    if len(str(row['StockCode'])) == 6 or len(str(row['StockCode'])) == 7 :
        df.at[i, 'ProductAttribute'] = str(row['StockCode'])[5:]

        if (str(row['StockCode'])[0:5]).isnumeric(): 
             df.at[i, 'StockCode'] = int(str(row['StockCode'])[0:5])
        else:
            row['StockCode'] = str(row['StockCode'] + 'XXXXXXXXXXXXXXXXX')

In [22]:
for i, r in df.iterrows(): 
    if len(str(row['StockCode'])) == 5:
        if str(row['StockCode']).isnumeric():
            pass
        else:
            print(row['StockCode'])
# seems fine

In [23]:
temp = pd.DataFrame()
temp['code'] = df['StockCode']
temp['desc'] = df['Description']
temp['len'] = None
for index, row in temp.iterrows(): 
    temp['len'][index] = len(str(row['code']))
temp.groupby('len').agg(['count', 'nunique'])

code             desc        
       count nunique    count nunique
len                                  
4         18       1       18       1
5    1021128    3946  1021128    5374
6          2       1        2       1
8         99      14       99      14
9         25       2       25       2
13      5506       4     5506      14

the rest are fine and it seems normal.

5 char is for products and 13 char are other related record to the purchase.

In [24]:
DFCK3 = df.copy()
DFCK3.shape
#checkpoint 1

(1026778, 11)

### Quantity

- We have negative quantity which is basically impossible for 22,950

In [25]:
df['Quantity'].describe()

count   1026778.000
mean         10.449
std         168.654
min      -80995.000
25%           1.000
50%           3.000
75%          10.000
max       80995.000
Name: Quantity, dtype: float64

In [26]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'IsCanceled', 'TotalPrice',
       'ProductAttribute'],
      dtype='object')

In [27]:
df[['Quantity', 'UnitPrice','TotalPrice', 'IsCanceled']].groupby('IsCanceled').agg(['min', 'max']).T

IsCanceled          False       True 
Quantity   min      1.000  -80995.000
           max  80995.000       1.000
UnitPrice  min      0.001       0.010
           max  25111.090   38970.000
TotalPrice min      0.001 -168469.600
           max 168469.600     373.570

- positive quantity in canceled order! 
- positive price in canceled order! 
gotta fix them: 

In [28]:
df.loc[ (df['IsCanceled'] == True)  & (df['Quantity'] > 0), : ]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,IsCanceled,TotalPrice,ProductAttribute
618708,496350,Manual_______,Manual,1,2010-02-01 08:24:00,373.570,NaN,United Kingdom,True,373.570,None


In [29]:
#Just one row, fix it by hand :) 
df.loc[ (df['IsCanceled'] == True)  & (df['Quantity'] > 0), 'Quantity' ] = -1
df.loc[ (df['IsCanceled'] == True)  & (df['TotalPrice'] > 0), 'TotalPrice' ] = -373.570

### Invoice Date

In [30]:
df['InvoiceDate'].info()

<class 'pandas.core.series.Series'>
Index: 1026778 entries, 0 to 1044846
Series name: InvoiceDate
Non-Null Count    Dtype         
--------------    -----         
1026778 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 47.9 MB


In [31]:
(df['InvoiceDate'].min()).date() , (df['InvoiceDate'].max()).date()

(datetime.date(2009, 12, 1), datetime.date(2011, 12, 9))

it's pretty fine! we'll needed more column and info which I'll add after data cleaning phase. 

### Country

In [32]:
df['Country'].isnull().sum()

0

In [33]:
df[['Country', 'InvoiceNo']].groupby('Country').count()

,InvoiceNo
Country,
Australia,1887
Austria,938
Bahrain,126
Belgium,3109
Bermuda,34
Brazil,94
Canada,228
Channel Islands,1646
Cyprus,1157


we have 751 null values! 
also there is a redunduncies, as there are seperated eauropian country and also EU as a whole. Also data set is not balanced and most of data is on England, so I just ignore the problem in this area and I'll investigate more on aggregated dataframe coming next

### Customer ID

Almost 25% of this data is Null! 

In [34]:
df['CustomerID'].isnull().sum()/df.shape[0]

0.2230842499547127

In [35]:
TempCheck = df.groupby('InvoiceNo').agg(lambda x: x.nunique())

Check_stats = pd.DataFrame({'Mean': TempCheck.mean(), 'Min': TempCheck.min(), 'Max': TempCheck.max()})
Check_stats

,Mean,Min,Max
StockCode,20.475,1,1075
Description,21.056,1,1108
Quantity,5.206,1,90
InvoiceDate,1.002,1,2
UnitPrice,8.214,1,63
CustomerID,0.929,0,1
Country,1.000,1,1
IsCanceled,1.000,1,1
TotalPrice,11.928,1,239
ProductAttribute,1.523,0,24


So the result is showing that: 

for InoviceDate we have a problem with several rows as the max of unique dates are two. I'll take the most repeated one as aggregated value. 

for customer ID we have either null of one. so I'll get the one. 

for country and IsCanceled we have no problem.

==> building the aggregated orders dataset in order to impute some mmissing Customer IDs


## Aggregate data to create Order

In [36]:
templist = ['InvoiceNo', 'StockCode', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'TotalPrice', 'IsCanceled']

Orders = df[templist].groupby('InvoiceNo').agg(  ItemsList = ('StockCode', lambda x : list(x)),
                                        TotalNumberofItems = ('Quantity', 'sum'),
                                        InvoiceDate = ('InvoiceDate', lambda x: x.mode()[0] if not x.mode().empty else None),
                                        AverageItemPrice = ('UnitPrice', 'mean'),
                                        CustomerID = ('CustomerID', lambda x: x.mode()[0] if not x.mode().empty else None), 
                                        Country = ('Country', 'first'), 
                                        TotalOrderPrice = ('TotalPrice', 'sum'),
                                        IsCanceled = ('IsCanceled', 'first')
                                    )
Orders.head()

,ItemsList,TotalNumberofItems,InvoiceDate,AverageItemPrice,CustomerID,Country,TotalOrderPrice,IsCanceled
InvoiceNo,,,,,,,,
489434,"[85048, 79323, 79323, 22041, 21232, 22064, 218...",166,2009-12-01 07:45:00,4.081,13085.000,United Kingdom,505.300,False
489435,"[22350, 22349, 22195, 22353]",60,2009-12-01 07:46:00,2.625,13085.000,United Kingdom,145.800,False
489436,"[48173, 21755, 21754, 84879, 22119, 22142, 222...",193,2009-12-01 09:06:00,3.731,13078.000,United Kingdom,630.330,False
489437,"[22143, 22145, 22130, 21364, 21360, 21351, 213...",145,2009-12-01 09:08:00,3.628,15362.000,United Kingdom,310.750,False
489438,"[21329, 21252, 21100, 21033, 20711, 21410, 214...",826,2009-12-01 09:24:00,2.591,18102.000,United Kingdom,2286.240,False


In [37]:
Orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48195 entries, 489434 to 581587
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ItemsList           48195 non-null  object        
 1   TotalNumberofItems  48195 non-null  int64         
 2   InvoiceDate         48195 non-null  datetime64[ns]
 3   AverageItemPrice    48195 non-null  float64       
 4   CustomerID          44776 non-null  float64       
 5   Country             48195 non-null  object        
 6   TotalOrderPrice     48195 non-null  float64       
 7   IsCanceled          48195 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 3.0+ MB


In [38]:
print('orders table null percents:')
Orders.isnull().sum()/Orders.shape[0]*100

orders table null percents:


ItemsList            0.000
TotalNumberofItems   0.000
InvoiceDate          0.000
AverageItemPrice     0.000
CustomerID           7.094
Country              0.000
TotalOrderPrice      0.000
IsCanceled           0.000
dtype: float64

In [39]:
Orders.describe()

,TotalNumberofItems,InvoiceDate,AverageItemPrice,CustomerID,TotalOrderPrice
count,48195.000,48195,48195.000,44776.000,48195.000
mean,222.604,2010-12-20 16:10:49.417989376,20.667,15265.061,399.005
min,-87167.000,2009-12-01 07:45:00,0.010,12346.000,-168469.600
25%,18.000,2010-06-23 12:37:00,2.085,13777.000,58.200
50%,109.000,2010-11-28 11:04:00,3.003,15164.000,232.580
75%,248.000,2011-07-01 13:55:30,4.600,16754.000,427.800
max,87167.000,2011-12-09 12:50:00,38970.000,18287.000,168469.600
std,1273.390,NaN,349.147,1719.414,1643.100


In [40]:
CleanedOrders = Orders.dropna()

In [41]:
CleanedOrders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44776 entries, 489434 to 581587
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ItemsList           44776 non-null  object        
 1   TotalNumberofItems  44776 non-null  int64         
 2   InvoiceDate         44776 non-null  datetime64[ns]
 3   AverageItemPrice    44776 non-null  float64       
 4   CustomerID          44776 non-null  float64       
 5   Country             44776 non-null  object        
 6   TotalOrderPrice     44776 non-null  float64       
 7   IsCanceled          44776 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 2.8+ MB


I also checked if I can find any CustomerIDs after aggregation of orders, which wasn't succesful: 

In [42]:
#check if we can fill some Customer ID now: 
df.join(Orders, on ='InvoiceNo', how='left' , rsuffix='__Orders').isnull().sum()

InvoiceNo                   0
StockCode                   0
Description                 0
Quantity                    0
InvoiceDate                 0
UnitPrice                   0
CustomerID             229058
Country                     0
IsCanceled                  0
TotalPrice                  0
ProductAttribute       903413
ItemsList                   0
TotalNumberofItems          0
InvoiceDate__Orders         0
AverageItemPrice            0
CustomerID__Orders     229058
Country__Orders             0
TotalOrderPrice             0
IsCanceled__Orders          0
dtype: int64

## Output Data

In [43]:
#Save the Order
Orders.to_csv('./Data/Orders.CSV')
CleanedOrders.to_csv('./Data/CleanedOrders.CSV')

In [44]:
#Saving all data
df.to_csv('./Data/CleanedOrderItemRecords_.csv',  index=False, compression='gzip')
NonNulldf = df.dropna(subset=['CustomerID'])
NonNulldf.to_csv('./Data/NonNullOrderItemRecords_.csv',  index=False, compression='gzip')